In [3]:
"""
Simple script to download European yearly electricity data from Ember Energy API
"""

import requests
import json

# Your API key
API_KEY = "c29fab08-15dd-1932-591a-d4c049881e2c"

# API base URL
BASE_URL = "https://api.ember-energy.org"

def download_europe_yearly_data():
    """
    Download yearly electricity data for Europe
    """
    # Endpoint for yearly generation data
    url = f"{BASE_URL}/electricity/generation/yearly"
    
    # Headers with authentication
    headers = {
        "api-key": API_KEY
    }
    
    # Parameters - filter for Europe
    params = {
        "entity": "Europe",
        "start_date": "2020",  # Adjust as needed
        "end_date": "2024"     # Adjust as needed
    }
    
    print(f"Requesting data from: {url}")
    print(f"Parameters: {params}")
    print("-" * 60)
    
    try:
        # Make the API request
        response = requests.get(url, headers=headers, params=params)
        
        # Check if request was successful
        if response.status_code == 200:
            data = response.json()
            
            # Save to JSON file
            output_file = "europe_yearly_electricity_data.json"
            with open(output_file, 'w') as f:
                json.dump(data, f, indent=2)
            
            print(f"✓ Success! Data saved to {output_file}")
            print(f"  Total records: {len(data.get('data', []))}")
            
            return data
        
        elif response.status_code == 401:
            print("✗ Authentication failed!")
            print("  Please verify your API key is correct.")
            
        elif response.status_code == 404:
            print("✗ Endpoint not found!")
            print("  The API endpoint may have changed.")
            print("  Visit https://api.ember-climate.org/docs for updated documentation.")
            
        else:
            print(f"✗ Request failed with status code: {response.status_code}")
            print(f"  Response: {response.text}")
            
    except requests.exceptions.ConnectionError:
        print("✗ Connection error!")
        print("  Unable to connect to the API. Check your internet connection.")
        
    except Exception as e:
        print(f"✗ An error occurred: {str(e)}")
    
    return None

if __name__ == "__main__":
    print("=" * 60)
    print("Ember Energy API - European Yearly Electricity Data")
    print("=" * 60)
    print()
    
    # Download the data
    data = download_europe_yearly_data()
    
    if data:
        print()
        print("Sample of the data:")
        print(json.dumps(data, indent=2)[:500] + "...")
    
    print()
    print("=" * 60)
    print("For API documentation, visit:")
    print("https://api.ember-climate.org/docs")
    print("=" * 60)

Ember Energy API - European Yearly Electricity Data

Requesting data from: https://api.ember-energy.org/electricity/generation/yearly
Parameters: {'entity': 'Europe', 'start_date': '2020', 'end_date': '2024'}
------------------------------------------------------------
✗ Endpoint not found!
  The API endpoint may have changed.
  Visit https://api.ember-climate.org/docs for updated documentation.

For API documentation, visit:
https://api.ember-climate.org/docs


In [4]:
import requests

my_api_key = "c29fab08-15dd-1932-591a-d4c049881e2c"
base_url = "https://api.ember-energy.org"
query_url = (
  f"{base_url}/v1/electricity-generation/yearly"
  + f"?entity_code=BRA&is_aggregate_series=false&start_date=2000&api_key={my_api_key}"
)

response = requests.get(query_url)

if response.status_code == 200:
    data = response.json()

In [6]:
print("Top-level type:", type(raw_1))
print("raw length:", len(raw_1))
print("metadata keys:", list(raw_1[0].keys()))
print("first record keys:", list(raw_1[1][0].keys()))

df_preview_1 = pd.DataFrame(raw_1[1])
print("DataFrame columns:", df_preview_1.columns.tolist())

df_preview_1.head(10)

NameError: name 'raw_1' is not defined

In [ ]:
"""
Comprehensive script to download yearly electricity data from Ember Energy API
API Documentation: https://api.ember-energy.org/v1/docs
Based on official documentation
"""

import requests
import json
import pandas as pd
from typing import Optional, Dict, Any

# API Configuration
API_KEY = "c29fab08-15dd-1932-591a-d4c049881e2c"
BASE_URL = "https://api.ember-energy.org/v1"

class EmberAPI:
    """Client for Ember Energy API"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = BASE_URL
    
    def get_yearly_generation(self, 
                            entity_code: Optional[str] = None,
                            start_date: Optional[int] = None,
                            end_date: Optional[int] = None,
                            is_aggregate_series: Optional[bool] = None,
                            series: Optional[str] = None) -> Optional[Dict[Any, Any]]:

        endpoint = f"{self.base_url}/electricity-generation/yearly"
        
        params = {"api_key": self.api_key}
        if entity_code:
            params['entity_code'] = entity_code
        if start_date:
            params['start_date'] = start_date
        if end_date:
            params['end_date'] = end_date
        if is_aggregate_series is not None:
            params['is_aggregate_series'] = str(is_aggregate_series).lower()
        if series:
            params['series'] = series
        
        return self._make_request(endpoint, params)
    
    def get_monthly_generation(self,
                              entity_code: Optional[str] = None,
                              start_date: Optional[str] = None,
                              end_date: Optional[str] = None,
                              is_aggregate_series: Optional[bool] = None) -> Optional[Dict[Any, Any]]:
        """
        Get monthly electricity generation data
        
        Parameters:
        - entity_code: Country/region code
        - start_date: Start date in YYYY-MM format (e.g., "2023-01")
        - end_date: End date in YYYY-MM format
        - is_aggregate_series: True for aggregate data, False to exclude
        """
        endpoint = f"{self.base_url}/electricity-generation/monthly"
        
        params = {"api_key": self.api_key}
        if entity_code:
            params['entity_code'] = entity_code
        if start_date:
            params['start_date'] = start_date
        if end_date:
            params['end_date'] = end_date
        if is_aggregate_series is not None:
            params['is_aggregate_series'] = str(is_aggregate_series).lower()
        
        return self._make_request(endpoint, params)
    
    def get_options(self, dataset: str, filter_name: str) -> Optional[Dict[Any, Any]]:
        """
        Get available options for a dataset parameter
        
        Parameters:
        - dataset: Dataset name (e.g., "electricity-generation/yearly")
        - filter_name: Filter to get options for (e.g., "entity_code", "series", "date")
        
        Examples:
        - get_options("electricity-generation/yearly", "entity_code") - list of countries
        - get_options("electricity-generation/yearly", "series") - list of fuel types
        - get_options("electricity-generation/monthly", "date") - available dates
        """
        endpoint = f"{self.base_url}/options/{dataset}/{filter_name}"
        params = {"api_key": self.api_key}
        
        return self._make_request(endpoint, params)
    
    def _make_request(self, endpoint: str, params: Dict) -> Optional[Dict[Any, Any]]:
        """Make API request"""
        try:
            response = requests.get(endpoint, params=params)
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 403:
                print(f"Authentication Error (403): Check your API key")
                print(f"Response: {response.text}")
            elif response.status_code == 404:
                print(f"Not Found (404): Check the endpoint URL")
                print(f"Response: {response.text}")
            else:
                print(f"Error {response.status_code}: {response.text}")
            
            return None
                
        except Exception as e:
            print(f"Request failed: {str(e)}")
            return None

def save_data(data: Any, filename: str, format: str = 'both'):
    """Save data to file(s)"""
    if not data:
        print("No data to save")
        return
    
    # Handle different data structures
    if isinstance(data, list):
        records = data
    elif isinstance(data, dict):
        if 'data' in data:
            records = data['data']
        elif 'results' in data:
            records = data['results']
        else:
            records = [data]
    else:
        records = [data]
    
    # Save JSON
    if format in ['json', 'both']:
        json_file = filename if filename.endswith('.json') else f"{filename}.json"
        with open(json_file, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"✓ Saved to {json_file}")
    
    # Save CSV
    if format in ['csv', 'both'] and records:
        try:
            df = pd.DataFrame(records)
            csv_file = filename.replace('.json', '.csv') if filename.endswith('.json') else f"{filename}.csv"
            df.to_csv(csv_file, index=False)
            print(f"✓ Saved to {csv_file} ({len(records)} records)")
        except Exception as e:
            print(f"Could not save CSV: {e}")

# European country codes (ISO 3166-1 alpha-3)
EUROPEAN_COUNTRIES = {
    "AUT": "Austria", "BEL": "Belgium", "BGR": "Bulgaria", 
    "HRV": "Croatia", "CZE": "Czech Republic", "DNK": "Denmark",
    "EST": "Estonia", "FIN": "Finland", "FRA": "France",
    "DEU": "Germany", "GRC": "Greece", "HUN": "Hungary",
    "IRL": "Ireland", "ITA": "Italy", "LVA": "Latvia",
    "LTU": "Lithuania", "NLD": "Netherlands", "POL": "Poland",
    "PRT": "Portugal", "ROU": "Romania", "SVK": "Slovakia",
    "SVN": "Slovenia", "ESP": "Spain", "SWE": "Sweden",
    "GBR": "United Kingdom", "NOR": "Norway", "CHE": "Switzerland"
}

if __name__ == "__main__":
    print("=" * 70)
    print("Ember Energy API - Yearly Electricity Data Downloader")
    print("=" * 70)
    print()
    
    # Initialize API client
    client = EmberAPI(API_KEY)
    
    # Example 1: Get available country codes
    print("1. Getting list of available countries...")
    options = client.get_options("electricity-generation/yearly", "entity_code")
    if options:
        print(f"✓ Found {len(options.get('data', []))} countries/regions")
        print()
    
    # Example 2: Get Europe aggregate data
    print("2. Downloading yearly generation data for Europe (2020-2024)...")
    europe_data = client.get_yearly_generation(
        entity_code="EUR",
        start_date=2020,
        end_date=2024,
        is_aggregate_series=True
    )
    if europe_data:
        save_data(europe_data, "europe_yearly_generation")
    print()
    
    # Example 3: Get data for specific European countries
    print("3. Downloading data for Germany, France, and Italy...")
    multi_country_data = client.get_yearly_generation(
        entity_code="DEU,FRA,ITA",  # Multiple countries
        start_date=2020,
        end_date=2024,
        is_aggregate_series=False
    )
    if multi_country_data:
        save_data(multi_country_data, "europe_countries_yearly")
    print()
    
    # Example 4: Get renewable energy data for Europe
    print("4. Downloading wind and solar data for Europe...")
    renewables_data = client.get_yearly_generation(
        entity_code="EUR",
        start_date=2020,
        end_date=2024,
        series="Wind,Solar",  # Multiple series
        is_aggregate_series=False
    )
    if renewables_data:
        save_data(renewables_data, "europe_renewables_yearly")
    print()
    
    # Example 5: Get data for all European countries individually
    print("5. Downloading data for top 5 EU countries individually...")
    top_countries = ["DEU", "FRA", "ITA", "ESP", "POL"]
    
    for code in top_countries:
        country_name = EUROPEAN_COUNTRIES.get(code, code)
        print(f"   - {country_name} ({code})...")
        
        country_data = client.get_yearly_generation(
            entity_code=code,
            start_date=2020,
            end_date=2024,
            is_aggregate_series=False
        )
        
        if country_data:
            save_data(country_data, f"{code.lower()}_yearly", format='csv')


Ember Energy API - Yearly Electricity Data Downloader

1. Getting list of available countries...
✓ Found 0 countries/regions

2. Downloading yearly generation data for Europe (2020-2024)...
✓ Saved to europe_yearly_generation.json

3. Downloading data for Germany, France, and Italy...
✓ Saved to europe_countries_yearly.json
✓ Saved to europe_countries_yearly.csv (150 records)

4. Downloading wind and solar data for Europe...
✓ Saved to europe_renewables_yearly.json

5. Downloading data for top 5 EU countries individually...
   - Germany (DEU)...
✓ Saved to deu_yearly.csv (50 records)
   - France (FRA)...
✓ Saved to fra_yearly.csv (50 records)
   - Italy (ITA)...
✓ Saved to ita_yearly.csv (50 records)
   - Spain (ESP)...
✓ Saved to esp_yearly.csv (50 records)
   - Poland (POL)...
✓ Saved to pol_yearly.csv (50 records)

Download complete!

Tips:
  - Use entity_code='DEU,FRA,ITA' for multiple countries
  - Use series='Wind,Solar' for specific fuel types
  - Set is_aggregate_series=False t